In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read the data
artistas = pd.read_csv('Artistas.csv', sep=',')
artistas = artistas.drop_duplicates()

entradas = pd.read_csv('Entradas Cortesia.csv', sep=',')
entradas = entradas.drop_duplicates()

eventos = pd.read_csv('Eventos.csv', sep=',')
eventos = eventos.drop_duplicates()

hosp_traslados = pd.read_csv('Hospedajes y Traslados.csv', sep=',')
hosp_traslados = hosp_traslados.drop_duplicates()

productoras = pd.read_csv('Productoras.csv', sep=',')
productoras = productoras.drop_duplicates()

tour = pd.read_csv('Tour.csv', sep=',')
tour = tour.drop_duplicates()


artistas = artistas.rename(columns={'Nombre artistico': 'Artista'})
entradas = entradas.rename(columns={'evento': 'Evento'})
entradas = entradas.rename(columns={'artista': 'Nombre'})

In [3]:
artistas['ID_artista'] = pd.Series(range(1, len(artistas) + 1), index=artistas.index)
artistas['ID_artista'] = artistas['ID_artista'].apply(lambda x: 'A' + str(x))

entradas['ID_entrada'] = pd.Series(range(1, len(entradas) + 1), index=entradas.index)
entradas['ID_entrada'] = entradas['ID_entrada'].apply(lambda x: 'E' + str(x))

eventos['ID_evento'] = pd.Series(range(1, len(eventos) + 1), index=eventos.index)
eventos['ID_evento'] = eventos['ID_evento'].apply(lambda x: 'E' + str(x))

productoras['ID_productora'] = pd.Series(range(1, len(productoras) + 1), index=productoras.index)
productoras['ID_productora'] = productoras['ID_productora'].apply(lambda x: 'P' + str(x))

tour['ID_tour'] = pd.Series(range(1, len(tour) + 1), index=tour.index)
tour['ID_tour'] = tour['ID_tour'].apply(lambda x: 'T' + str(x))

hosp_traslados['ID_hosp_traslado'] = pd.Series(range(1, len(hosp_traslados) + 1), index=hosp_traslados.index)
hosp_traslados['ID_hosp_traslado'] = hosp_traslados['ID_hosp_traslado'].apply(lambda x: 'HT' + str(x))

In [4]:
artistas = artistas.rename(columns={'Artista': 'Nombre', 'id': 'ID_artista', 'Inicio carrera': 'Fecha_inicio', 'Nº contacto': 'Contacto'})
artistas['Artista'] = artistas['Nombre'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
artistas = artistas.drop(['Artista'], axis=1)
entradas = entradas.merge(artistas[['Nombre', 'ID_artista']], on='Nombre', how='left')
# poner en minuscula y quitar acentos a todas las columnas de tipo string de la tabla eventos
eventos['Evento'] = eventos['Evento'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# cambiar nombre  artistas a Artista en artistas
artistas = artistas.rename(columns={'Nombre': 'Artista'})
eventos = eventos.rename(columns={'Fecha de incio': 'Fecha_inicio'})
entradas = entradas.rename(columns={'Nombre': 'Artista'})
entradas = entradas.drop(['ID_artista'], axis=1)
productoras = productoras.rename(columns={'País': 'Pais', 'Nº contacto': 'Contacto'})
hosp_traslados = hosp_traslados.rename(columns={'fecha inicio': 'fecha_inicio', 'fecha termino': 'fecha_termino', 'hora llegada': 'hora_llegada', 'codigo reserva': 'codigo_reserva', 'nombre hotel': 'nombre_hotel', 'tipo traslado': 'tipo_traslado',
'aeropuerto salida': 'aeropuerto_salida', 'hora salida': 'hora_salida', 'aeropuerto llegada': 'aeropuerto_llegada', 'hora llegada': 'hora_llegada', 'ID_hosp_traslado': 'ID_ht'})
tour = tour.rename(columns={'fecha inicio': 'fecha_inicio', 'fecha termino': 'fecha_termino'})

In [5]:
# En eventos fecha de inicio, reemplazar las que estan escritas dd-mm-aaaa por dd/mm/aaaa
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].str.replace('-', '/')
# en las fechas, reemplazar los numeros que están escritos como 1 por 02. No se puede hacer con un replace porque hay fechas que tienen 1 y otras que tienen 01
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].apply(lambda x: x.replace('/1/', '/01/'))
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].apply(lambda x: x.replace('/2/', '/02/'))
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].apply(lambda x: x.replace('/3/', '/03/'))
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].apply(lambda x: x.replace('/4/', '/04/'))
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].apply(lambda x: x.replace('/5/', '/05/'))
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].apply(lambda x: x.replace('/6/', '/06/'))
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].apply(lambda x: x.replace('/7/', '/07/'))
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].apply(lambda x: x.replace('/8/', '/08/'))
eventos['Fecha_inicio'] = eventos['Fecha_inicio'].apply(lambda x: x.replace('/9/', '/09/'))

hosp_traslados['fecha_inicio'] = hosp_traslados['fecha_inicio'].str.replace('-', '/')


In [6]:
from datetime import datetime
# convertir las fechas de los hosp y traslados a datetime
hosp_traslados['fecha_inicio'] = hosp_traslados['fecha_inicio'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
# convertir fecha_inicio a dia/mes/año
hosp_traslados['fecha_inicio'] = hosp_traslados['fecha_inicio'].apply(lambda x: x.strftime('%d/%m/%Y'))

# hacer lo mismo con fecha_termino
hosp_traslados['fecha_termino'] = hosp_traslados['fecha_termino'].str.replace('-', '/')
hosp_traslados['fecha_termino'] = hosp_traslados['fecha_termino'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))
hosp_traslados['fecha_termino'] = hosp_traslados['fecha_termino'].apply(lambda x: x.strftime('%d/%m/%Y'))


In [7]:
# Transformar hora_llegada y hora_salida a string.
hosp_traslados['hora_llegada'] = hosp_traslados['hora_llegada'].astype(str)
hosp_traslados['hora_salida'] = hosp_traslados['hora_salida'].astype(str)
# Transforamr string nan a objeto nan
hosp_traslados['hora_llegada'] = hosp_traslados['hora_llegada'].apply(lambda x: np.nan if x == 'nan' else x)
hosp_traslados['hora_salida'] = hosp_traslados['hora_salida'].apply(lambda x: np.nan if x == 'nan' else x)

# convertir hora_llegada y hora_salida a datetime, ignorando los nan
# usar formato de 24 horas
hosp_traslados['hora_llegada'] = hosp_traslados['hora_llegada'].apply(lambda x: datetime.strptime(x, '%H:%M') if pd.notnull(x) else x)
hosp_traslados['hora_salida'] = hosp_traslados['hora_salida'].apply(lambda x: datetime.strptime(x, '%H:%M') if pd.notnull(x) else x)

# quitar cadena 1900-01-01 de hora_llegada y hora_salida
hosp_traslados['hora_llegada'] = hosp_traslados['hora_llegada'].apply(lambda x: x.strftime('%H:%M') if pd.notnull(x) else x)
hosp_traslados['hora_salida'] = hosp_traslados['hora_salida'].apply(lambda x: x.strftime('%H:%M') if pd.notnull(x) else x)

In [8]:
entradas.to_csv('entradas_bdd.csv', index=False)
artistas.to_csv('artistas_bdd.csv', index=False)
eventos.to_csv('eventos_bdd.csv', index=False)
productoras.to_csv('productoras_bdd.csv', index=False)
hosp_traslados.to_csv('hosp_traslados_bdd.csv', index=False)
tour.to_csv('tour_bdd.csv', index=False)